## **1. Load File**

In [ ]:
# Load in latest snapshot of simulation "3D Model B"
# Huang, Yu, Lee, Dong, & Bai 2025

from rixin_rw import DataMasking

path = '/home/dasein/data/disk_planet_3D.cons.00400.athdf'
dataset = DataMasking(path)

## **2. Data Post-Processing**

In [ ]:
# (Optional) Interactive Phi Angle
#phiangle = float(input("Choose your ϕ angles [0, 2π)"))

# External Packages
import matplotlib.pyplot as plt
import pyridoxine.plt as rxplt

for phiangle in range(0, 360):
    rxplt.plt_params('paper')
    fig = plt.figure(figsize=(11,8.5))

    # Polar axes: [left, bottom, width, height]
    ax = fig.add_axes([0.2, 0.1, 0.7, 1.0], projection='polar')

    # Map degree phiangle -> discrete index along ccphi
    #i_phi = int(np.round(phiangle / 720.0 * dataset.ccphi.size)) % dataset.ccphi.size

    # Overplot levels 1 and 2
    for _lev in range (1, 3):
        dataset.get_level(_lev)

        # extract dust density at given phi (where the planet locates)
        Q = ds['rhod_1'][int(dataset.ccphi.size*phiangle//360), :, :]
        #Q = ds['rhod_1'][i_phi, :, :]
        im = axes.pcolormesh(
            dataset.cctheta,
            dataset.ccr,
            np.log10(Q).T,
            shading='auto',
            cmap="plasma",
            vmin=-8, vmax=0.1,
            rasterized=True
        )

    # set theta to increase in the clockwise direction
    axes.set_theta_direction(-1)
    # position θ=0 at North and draw a faint white grid
    axes.set_theta_zero_location('N')
    axes.grid(True, ls=':', alpha=0.3, c='w')
    
    # limit θ to ±5 scale heights around the midplane (in degrees)
    H = 0.05
    axes.set_thetamin(180/np.pi * (np.pi/2 - 5*H))
    axes.set_thetamax(180/np.pi * (np.pi/2 + 5*H))

    # set angular ticks at ±3H and 0, and label them accordingly
    axes.set_xticks([np.pi/2 - 3*H, np.pi/2, np.pi/2 + 3*H])
    axes.set_xticklabels([r"$3H$", r"$0$", r"$-3H$"])
    # pad the angular tick labels outward for readability
    axes.tick_params(axis='x', which='major', pad=24)

    # reduce radial ticks to only 0.5 and 1 for clarity
    axes.set_rticks([0.5, 1])

    # annotate the plot with φ location in blue, centered at given figure coordinates
    # fig.text(0.195, 0.645, fr"$\phi={phiangle:.2f}$", ha='center', va='center',
    fig.text(0.195, 0.645, fr"$\phi={phiangle:03d}^\circ$", ha='center', va='center',
             fontsize=30, color='blue')

    # add a new horizontal axes for the colorbar
    cax = fig.add_axes([0.125, 0.9, 0.75, 0.02])
    cbar = fig.colorbar(im, cax=cax, orientation='horizontal')
    # move the colorbar label to the top of the bar
    cbar.axes.xaxis.set_label_position('top')
    # set the colorbar label including simulation time and log dust density
    cbar.set_label(
        # fr"3D Model B, $t={dataset.t/(2*np.pi):.1f}"+r"P_{\rm orb}$, " + r"$\log \rho_{\rm d}$", labelpad=12)
        fr"3D Model B, $t={dataset.t/(2*np.pi):.1f}P_{{\rm orb}}$, $\log \rho_{{\rm d}}$",
        labelpad=12
    )

    filename = f"/home/dasein/repositories/notebooks_julian/disk_slices/plot_{phiangle:03d}.png"
    fig.savefig(filename)
    plt.close(fig)
